<a href="https://colab.research.google.com/github/franmazzaro/7506R-2C2023-GRUPO39/blob/main/7506R_TP2_GRUPO39_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librerias**

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# **Preprocesamiento**

In [49]:
df_criticas = pd.read_csv('/content/sample_data/train.csv')
df_criticas_test = pd.read_csv('/content/sample_data/test.csv')


In [50]:
# Preprocesamiento de texto
# Elimina caracteres especiales y convierte el texto a minúsculas
#df_criticas['review_es'] = df_criticas['review_es'].str.replace('[^\w\s]', '').str.lower()

In [51]:
# Tokenización y eliminación de stopwords
#nltk.download('stopwords')
#nltk.download('punkt')

#stop_words = set(stopwords.words('spanish'))

#df_criticas['review_es'] = df_criticas['review_es'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))


In [52]:
df_criticas.head(4)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo


Dividimos nuestro conjunto de train en 70 30

In [53]:
x_train, x_test, y_train, y_test = train_test_split(df_criticas.review_es,
                                                    df_criticas.sentimiento,
                                                    test_size=0.30,
                                                    random_state=25,
                                                    shuffle=True)

Antes de entrear un modelo para clasificacion necesitamos que este entrenamiento sea con valores numericos, y como nosotros tenemos palabras necesitamos convertirlas a vectores

In [54]:
model_NB = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_NB_2 = make_pipeline(CountVectorizer(), MultinomialNB())

model_NB.fit(x_train, y_train)
model_NB_2.fit(x_train, y_train)

categorias_predichas = model_NB.predict(x_test)
categorias_predichas_2 = model_NB_2.predict(x_test)


Utilizamos make_pipeline para aplicarle a nuestros modelos dos metodos distintos para vectorizar y luego el modelo de Naive Bayes Multidimensional. Una vez obtenidos los entrenamos

In [55]:
print(categorias_predichas)
print(categorias_predichas_2)

['positivo' 'positivo' 'positivo' ... 'negativo' 'positivo' 'positivo']
['positivo' 'positivo' 'positivo' ... 'negativo' 'positivo' 'positivo']


Estas son las metricas de los modelos con train

In [56]:
print(accuracy_score(y_test, categorias_predichas_2))
print(accuracy_score(y_test, categorias_predichas))
print(precision_score(y_test, categorias_predichas_2, average='micro'))
print(precision_score(y_test, categorias_predichas, average='micro'))

0.8376
0.8534666666666667
0.8376
0.8534666666666667


Prediccion de test

In [88]:
criticas_test = df_criticas_test['review_es']
categorias_predichas_test = model_NB.predict(criticas_test)


Metricas de prediccion de test

In [58]:
ids_test =  df_criticas_test.ID
filename = 'Naive_Bayes_Multidimensional'
prediccion = pd.DataFrame({"id": ids_test, "sentimiento": categorias_predichas_test})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)

# Conversion a bag of words

In [12]:
vectorizer = CountVectorizer()
reviews = list()
reviews = df_criticas['review_es'].tolist()
bag_of_words = vectorizer.fit_transform(reviews)
reviews_vectorizer = pd.DataFrame(bag_of_words.toarray()[:1000])
reviews_vectorizer.head()

,0,1,2,3,4,5,6,7,8,9,...,110373,110374,110375,110376,110377,110378,110379,110380,110381,110382
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
print(bag_of_words.toarray()[:5])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [14]:
vectorizer = TfidfVectorizer(smooth_idf = True)
fid = vectorizer.fit_transform(reviews)
print(fid.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Random Forest

In [59]:
# Vectorización de texto usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_criticas['review_es'])

# Codificar las etiquetas de sentimiento
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_criticas['sentimiento'])

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=25)

In [61]:
# Inicializa el clasificador RandomForest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenamiento del modelo
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [62]:
# Predicción en el conjunto de prueba
y_pred = rf_classifier.predict(X_test)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.8232666666666667
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.83      0.83      7599
           1       0.82      0.82      0.82      7401

    accuracy                           0.82     15000
   macro avg       0.82      0.82      0.82     15000
weighted avg       0.82      0.82      0.82     15000



In [63]:
X_test = vectorizer.transform(df_criticas_test['review_es'])

In [94]:
categorias_predichas_test_2 = rf_classifier.predict(X_test)

In [96]:
# Función para convertir valores
def convertir(valor):
    return "negativo" if valor == 0 else "positivo"

# Aplicar la función a cada elemento del array usando comprensión de listas
nuevo_array = [convertir(valor) for valor in categorias_predichas_test_2]

['negativo', 'negativo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'ne

In [97]:
ids_test =  df_criticas_test.ID
filename = 'Random_Forest_Prueba'
prediccion = pd.DataFrame({"id": ids_test, "sentimiento": nuevo_array})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)